# FiLM에 대하여

FiLM (Feature-wise Linear Modulation)은 딥 러닝에서 사용되는 테크닉으로, 특정 층(layer)의 특징(features)에 선형 변환을 적용하는 방식입니다. 이 방법은 네트워크가 외부 정보에 따라 동적으로 동작을 조정할 수 있도록 해줍니다.

## FiLM의 작동 원리:

FiLM은 간단한 선형 변환(스케일링 및 이동)을 신경망의 층 출력에 적용합니다. 이 변환은 외부 정보에 조건을 맞춰 수행되며, 이 정보는 네트워크의 다른 입력이나 네트워크에 의해 학습된 잠재적 표현일 수 있습니다.

FiLM 변환은 다음과 같이 정의됩니다:

 FiLM(x) = gamma * x + beta
 
여기서  x 는 층의 출력(특징 맵), gamma는 스케일링 인자, beta는 이동 인자입니다. gamma와 beta는 학습 가능한 매개변수이며, 보통 네트워크의 다른 부분, 종종 별도의 신경망에서 출력됩니다.


## FiLM의 효과:

FiLM의 핵심은 네트워크가 외부 정보에 기반하여 내부 표현을 특징별로 수정할 수 있도록 하는 것입니다. 이는 입력이 처리되어야 하는 방식이 맥락이나 외부 조건에 의해 크게 달라지는 작업(예: 멀티모달 학습, 조건부 생성)에서 특히 유용합니다.

## 멀티모달리티에서의 FiLM 사용 예시:

멀티모달 학습에서 FiLM은 다양한 모달리티(예: 텍스트, 이미지, 소리)의 데이터를 처리할 때 이 모달리티 간의 상호작용을 학습하는 데 사용될 수 있습니다. 예를 들어, 이미지에 대한 설명을 생성하는 작업에서, 텍스트 입력(설명)이 이미지 처리 네트워크에 FiLM을 통해 영향을 미칠 수 있습니다. 이 경우, 텍스트 정보는 이미지 특징을 조건부로 조정하여 더 정확하고 관련성 높은 이미지 설명을 생성하는 데 도움을 줍니다.

예를 들어, "해변에서 노는 개"라는 텍스트 입력이 주어졌을 때, FiLM을 사용하는 이미지 처리 네트워크는 이 정보를 기반으로 해변과 관련된 특징(예: 모래, 바다)과 개와 관련된 특징(예: 털, 모양)을 강조하여 이미지를 분석할 수 있습니다.

FiLM은 이처럼 다양한 종류의 데이터와 그것들의 상호작용을 효과적으로 학습하고 이해하는 데 중요한 역할을 할 수 있습니다.

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FiLM(nn.Module):
    def forward(self, x, gamma, beta):
        # gamma와 beta의 크기를 이미지의 tensor 형식에 맞게 조정(브로드캐스팅)합니다.
        gamma = gamma.unsqueeze(2).unsqueeze(3)
        print(gamma.shape)
        beta = beta.unsqueeze(2).unsqueeze(3)
        print(beta.shape)

        # FiLM 변환을 적용합니다.
        return gamma * x + beta
    
class ImageNetwork(nn.Module):
    def __init__(self):
        super(ImageNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3) 
        self.film = FiLM() 

    def forward(self, x, gamma, beta):
        x = F.relu(self.conv1(x)) # Conv2d 진행
        print(f'x_shape = {x.shape}') # Conv2d를 통과한 x의 shape ([1, 10, 62, 62])
        x = self.film(x, gamma, beta) # FiLM 레이어를 통과시켜 FiLM을 적용
        return x

class TextNetwork(nn.Module):
    def __init__(self, text_embedding_size, film_parameter_size):
        super(TextNetwork, self).__init__()
        self.fc = nn.Linear(text_embedding_size, film_parameter_size)

    def forward(self, text_embedding):
        film_params = self.fc(text_embedding) # text_embedding vector을 fully connect로 연결
        gamma, beta = film_params.chunk(2, dim=1) # chunk를 사용하여 film_params를 gamma와 beta로 나눈다.
        return gamma, beta



### 브로드캐스팅 과정

브로드캐스팅은 다음과 같은 과정을 통해 수행됩니다:

1. **차원 확장**: `gamma`와 `beta`가 `[1, 10, 1, 1]`에서 `[1, 10, 62, 62]`로 확장됩니다. 이는 PyTorch에서 자동으로 이루어지는 과정입니다.

2. **동일한 연산 적용**: 확장된 `gamma`와 `beta`는 이미지의 각 채널에 대해 모든 픽셀에 동일하게 적용됩니다.

### 수학적 연산

FiLM 연산인 `gamma * x + beta`는 다음과 같이 수행됩니다:

- 각 채널의 모든 픽셀 `x`에 대해, 동일한 채널의 `gamma` 값을 곱하고 `beta` 값을 더합니다.
- 결과적으로, 한 채널 내의 모든 픽셀은 동일한 `gamma`와 `beta` 값에 의해 조절됩니다.

### 결과

이러한 방식으로, FiLM 레이어는 입력 이미지의 각 채널에 대해 동일한 조절을 수행하게 됩니다. 이는 해당 채널의 전체 특성(예: 색상 조절, 밝기 조절 등)에 영향을 미치며, 멀티모달 시나리오에서 다른 입력(예: 텍스트)에 따라 이미지를 다르게 해석하는 데 도움을 줍니다.

In [45]:
# 네트워크 initialize
image_net = ImageNetwork()
text_net = TextNetwork(text_embedding_size=100, film_parameter_size=20) # example sizes

# 더미 인풋값 생성
dummy_image = torch.randn(1, 3, 64, 64) # Example image tensor
dummy_text_embedding = torch.randn(1, 100) # Example text embedding

# text_network에서 gamma, beta 값을 생성
gamma, beta = text_net(dummy_text_embedding)
print('gamma_shape = ',gamma.shape)
print('beta_shape = ',beta.shape)
# image_network에서 FiLM 적용
modulated_output = image_net(dummy_image, gamma, beta)
print(f'modulated_output_shape = {modulated_output.shape}')


gamma_shape =  torch.Size([1, 10])
beta_shape =  torch.Size([1, 10])
x_shape = torch.Size([1, 10, 62, 62])
torch.Size([1, 10, 1, 1])
torch.Size([1, 10, 1, 1])
modulated_output_shape = torch.Size([1, 10, 62, 62])


## 예시 학습 코드

data_loader에서 Image를 불러와서 학습하는 예시 코드를 작성해보면

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch import optim
from PIL import Image

# CustomDataset 클래스를 만듭니다.
class CustomDataset(Dataset):
    def __init__(self, image_paths, text_embeddings, targets):
        self.image_paths = image_paths
        self.text_embeddings = text_embeddings
        self.targets = targets

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 이미지를 불러오고 전처리합니다.
        image = Image.open(self.image_paths[idx])  # 이미지를 불러오고 resize후 Tensor로 변환합니다.
        image = image.resize(64,64)
        tf = transforms.ToTensor()
        img_t = tf(image)

        # text_embedding과 target을 가져옵니다.
        text_embedding = self.text_embeddings[idx]
        target = self.targets[idx]

        return img_t, text_embedding, target

text_embedding_size=100
film_parameter_size=20
output_channels = 10
learning_rate = 0.001

# 데이터셋 경로, text_embedding과, target 데이터를 준비합니다.
image_paths = ["image1.jpg", "image2.jpg", ...]  # 이미지 파일 경로 리스트
text_embeddings = torch.randn(len(image_paths), text_embedding_size)  # text_embedding
targets = torch.randn(len(image_paths), output_channels)  # target 데이터

# CustomDataset 인스턴스를 생성합니다.
custom_dataset = CustomDataset(image_paths, text_embeddings, targets)

# DataLoader를 설정합니다.
batch_size = 32  # 배치 크기 설정
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Criterion를 정의한다. (예를 들어, 평균 제곱 오차(MSELoss)를 사용)
criterion = nn.MSELoss()

# Optimizer를 정의한다. (예를 들어, Adam 옵티마이저를 사용)
optimizer = optim.Adam(list(image_net.parameters()) + list(text_net.parameters()), lr=learning_rate)

# Training 루프를 설정한다.
num_epochs = 10  # 학습 횟수 설정

for epoch in range(num_epochs):
    total_loss = 0.0

    # Dataloader로부터 이미지와 텍스트 데이터를 불러온다.
    for i, (images, text_embeddings, targets) in enumerate(data_loader):
        # image와 text_embedding를 GPU에 넘긴다
        images = images.to(device)
        text_embeddings = text_embeddings.to(device)

        # text_net를 통해 gamma와 beta를 얻는다.
        gamma, beta = text_net(text_embeddings)

        # image_net를 통해 이미지에 FiLM 레이어를 적용한다.
        output = image_net(images, gamma, beta)

        # Loss를 계산한다.
        loss = criterion(output, target)  # target은 FiLM 레이어를 적용한 이미지의 실제 값이어야 함

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 현재 epoch에서의 평균 loss 출력
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(data_loader)}')